# Microsoft Azure Data Manager for Agriculture and NDVI summary workflows into a single custom workflow

In this notebook, we will explain how to connect FarmVibes.AI with [Microsoft Azure Data Manager for Agriculture](https://aka.ms/farmvibesDMA), and provide an example of how to leverage the FarmVibes.AI workflows using ADMAg inputs. We will demonstrate how to compose the ADMAg and NDVI summary workflows into a single custom workflow, and check the results for the user's agriculture field.

## Import required modules

In this example, we will use basic modules for data manipulation (e.g., pandas, yaml, and datetime) and FarmVibes.AI vibe_core modules (client, and data).

In [ ]:
import yaml
import pandas as pd
from datetime import datetime
from pprint import pprint

from vibe_core.client import get_default_vibe_client
from vibe_core.data import ADMAgSeasonalFieldInput

## Define Azure Data Manager for Agriculture entities

We will start by providing the parameters that specify the Azure Data Manager for Agriculture connection (e.g., seasonal field, boundary, and farmer identifiers). Please, check Microsoft Azure Data Manager for Agriculture [documentation](https://aka.ms/farmvibesDMA) to check how to obtain these fields.

In the next cell, we retrieve the `CLIENT_SECRET` variable from the `data-manager-ag-secret` registered on the FarmVibes.AI cluster. To create a new key on the cluster you may want to use the following command on project's root folder:

```
$ bash farmvibes-ai.sh add-secret data-manager-ag-secret <CLIENT_SECRET_VALUE>
```
A message like the following, should be prompted.

`secret/data-manager-ag-secret created`

More details about FarmVibes.AI secrets can be found on the [FarmVibes.AI secrets documentation](https://microsoft.github.io/farmvibes-ai/docfiles/markdown/SECRETS.html).

In [ ]:
WORKFLOW_NAME = "data_ingestion/admag/admag_seasonal_field"

BASE_URL = ""
CLIENT_ID = ""
CLIENT_SECRET = "@SECRET(eywa-secrets, data-manager-ag-secret)"
AUTHORITY = ""
DEFAULT_SCOPE = ""

FARMER_ID = ""
SEASONAL_FIELD_ID=""
BOUNDARY_ID=""

## Create Seasonal Field input

Azure Data Manager for Agriculture uses `farmer_id`, `seasonal_field_id`, and `boundary_id` to identify a crop during a given season. This triple will be used to create a DataVibe subclass `SeasonalFieldInformation` that contains farm-related operations (e.g., fertilization, harvest, tillage, planting, crop name) that is used as input to the workflow (`data_ingestion/admag/admag_seasonal_field`). 

In [ ]:
input_data = ADMAgSeasonalFieldInput(
    farmer_id=FARMER_ID,
    seasonal_field_id=SEASONAL_FIELD_ID,
    boundary_id=BOUNDARY_ID,
)

# Retrieve information from ADMAg

The following cells execute the workflow and retrieve the results from FarmVibes.AI cluster.

In [ ]:
client = get_default_vibe_client()

In [ ]:
client.document_workflow(WORKFLOW_NAME)

In [ ]:
run = client.run(
    WORKFLOW_NAME,
    "ADMAg Workflow",
    input_data={"admag_input": input_data},
    parameters={
        "base_url": BASE_URL,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "authority": AUTHORITY,
        "default_scope": DEFAULT_SCOPE
    }
)

run.monitor()

### Check Workflow Output

Using the workflow output, we can see what happened in the Farm during the season.

In [ ]:
run.output['seasonal_field'][0]

## Check field characteristics

Users may want to navigate through the `SeasonalFieldInformation` object to evaluate farming data.

In [ ]:
seasonal_field = run.output['seasonal_field'][0]

pprint(f"Seasonal field planting date: {seasonal_field.time_range[0].strftime('%Y/%m/%d')}")
pprint(f"Seasonal field harvest date: {seasonal_field.time_range[1].strftime('%Y/%m/%d')}")

pprint(seasonal_field.geometry)

# Composing Workflows

In the following cells, we will evaluate Normalized Vegetation Index (NDVI) for a field during the season, whose information is obtained from Azure Data Manager for Agriculture. To do so, we will use the `farm_ai/agriculture/ndvi_summary` workflow:

In [ ]:
client.document_workflow("farm_ai/agriculture/ndvi_summary")

## Create and execute the custom workflow

Observe the following workflow composed of two tasks (**admag_seasonal_field** and **ndvi_summary**) in the next cell. The first task creates the `SeasonalFieldInformation` object which contains the farming-related data from Azure Data Manager for Agriculture, whereas `ndvi_summary` calculates the NVDI index for this crop during the season (from planting to harvest).

Please, observe the edge mapping **admag_seasonal_field.seasonal_field** to **ndvi_summary.user_input**. This enables the `SeasonalFieldInformation` objects to be used for evaluating NDVI the the field.

In [ ]:
custom_workflow = """
name: custom_admag
sources:
  admag_input:
    - admag_seasonal_field.admag_input
sinks:
  seasonal_field: admag_seasonal_field.seasonal_field
  ndvi_summary: ndvi_summary.timeseries
parameters:
  base_url:
  client_id:
  client_secret:
  authority:
  default_scope:
tasks:
  admag_seasonal_field:
    workflow: data_ingestion/admag/admag_seasonal_field
    parameters:
      base_url: "@from(base_url)"
      client_id: "@from(client_id)"
      client_secret: "@from(client_secret)"
      authority: "@from(authority)"
      default_scope: "@from(default_scope)"
  ndvi_summary:
    workflow: farm_ai/agriculture/ndvi_summary
edges:
  - origin: admag_seasonal_field.seasonal_field
    destination:
      - ndvi_summary.user_input
"""

workflow_dict = yaml.safe_load(custom_workflow)

In [ ]:
run = client.run(
    workflow_dict,
    "Seasonal Field NDVI Evaluation",
    input_data={"admag_input": input_data},
    parameters={
        "base_url": BASE_URL,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "authority": AUTHORITY,
        "default_scope": DEFAULT_SCOPE,
    }
)

run.monitor()

For the sake of simplicity, we will print only the harvest information of seasonal field. However, the `SeasonalField` returned object may contain data related to fertilization, organic amendments, tillage, and others.

In [ ]:
seasonal_field = run.output['seasonal_field'][0]
pprint(seasonal_field.harvests)

Checking the NDVI summary results:

In [ ]:
local_path = run.output["ndvi_summary"][0].assets[0].path_or_url
df = pd.read_csv(local_path)
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.plot(x="date", y="mean", title="NDVI for SeasonalField", ylabel="NDVI Mean", grid=True)

Checking the results as a DataFrame:

In [ ]:
df